In [4]:
import os
import pymssql

import pandas as pd
import numpy as np
import locale

from dotenv import load_dotenv
from utils_ccd import get_connection
from docxtpl import DocxTemplate


load_dotenv()
conn = get_connection()

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

from num2words import num2words

from docx2pdf import convert

c:\Users\05911205424\Documents\Dev\ccd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import subprocess

def generate_pdf(doc_path, path):

    subprocess.call(['soffice',
                  '--headless',
                 '--convert-to',
                 'pdf',
                 '--outdir',
                 path,
                 doc_path])
    return doc_path

import os
import win32com.client

def generate_pdf_office(doc_path: str, output_dir: str) -> str:
    # Ensure paths are absolute
    doc_path = os.path.abspath(doc_path)
    output_dir = os.path.abspath(output_dir)
    pdf_path = os.path.join(output_dir, os.path.splitext(os.path.basename(doc_path))[0] + '.pdf')

    word = win32com.client.Dispatch("Word.Application")
    word.Visible = False

    try:
        doc = word.Documents.Open(doc_path)
        doc.SaveAs(pdf_path, FileFormat=17)  # 17 = wdFormatPDF
        doc.Close()
    finally:
        word.Quit()

    return pdf_path

In [6]:
descontos_sql = ''' 
SELECT 
    pro.numero_processo,
    pro.ano_processo,
    pro.assunto,
    pm.Descricao as descricao,
    CONCAT(gp.Nome, ' (CPF: ', gp.Documento, ')') as nome,
    processo.dbo.fn_Exe_RetornaValorAtualizado(ed.IdDebito) as valor_multa,
    orgao_info.nome_orgao,
    ed.CodigoStatusDivida as status_divida
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
    ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm 
    ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed 
    ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
    ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp 
    ON gp.IdPessoa = edp.IDPessoa
OUTER APPLY (
    SELECT TOP 1 vspfr.nome_orgao
    FROM BdDIP.dbo.vwSiaiPessoalFolhaResumida vspfr
    WHERE vspfr.cpf = gp.Documento COLLATE SQL_Latin1_General_CP1_CI_AS
    ORDER BY vspfr.ano DESC, vspfr.mes DESC
) AS orgao_info
WHERE 
    pro.setor_atual = 'CCD'
    AND pm.Descricao = 'Protesto Efetivo Maio/2025'
    AND ed.CodigoTipoDebito NOT IN (1, 3)
'''
processos_descontos = pd.read_sql(descontos_sql, conn)

In [8]:
processos_descontos = processos_descontos[~processos_descontos.nome_orgao.isna()]

In [5]:
def criar_valor_extenso(x):
        if x and x > 1:
            f = lambda c: num2words(int(c), lang='pt_BR')
            return ' reais e '.join([f(y) for y in str('{:.2f}'.format(x)).split('.')]) + ' centavos'
        else:
            return ''

def converter_valor(x):
    if x:
        return locale.currency(x, grouping=True) + ' (' + criar_valor_extenso(x) + ')'
    else:
        return 'R$ 0,00 (zero)'

In [6]:
processos_descontos.fillna(0, inplace=True)

In [7]:
processos_descontos['processo'] = processos_descontos['numero_processo'] + '/' + processos_descontos['ano_processo'].astype(str)
processos_descontos['valor'] = processos_descontos['valor_multa'].apply(converter_valor)  

In [8]:
descontos_aberto = processos_descontos[processos_descontos.status_divida == 1]

In [9]:
descontos_aberto

,numero_processo,ano_processo,assunto,descricao,nome,valor_multa,nome_orgao,status_divida,processo,valor
0,000730,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,Protesto Efetivo Maio/2025,HUMBERTO HENRIQUE DE ANDRADE PEREIRA (CPF: 791...,16582.1711,PREFEITURA MUNICIPAL DE CAIÇARA DO RIO DO VENTO,1,000730/2025,"R$ 16.582,17 (dezesseis mil, quinhentos e oite..."
1,000730,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,Protesto Efetivo Maio/2025,HUMBERTO HENRIQUE DE ANDRADE PEREIRA (CPF: 791...,2802.3327,PREFEITURA MUNICIPAL DE CAIÇARA DO RIO DO VENTO,1,000730/2025,"R$ 2.802,33 (dois mil, oitocentos e dois reais..."
2,000730,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,Protesto Efetivo Maio/2025,HUMBERTO HENRIQUE DE ANDRADE PEREIRA (CPF: 791...,718.2909,PREFEITURA MUNICIPAL DE CAIÇARA DO RIO DO VENTO,1,000730/2025,"R$ 718,29 (setecentos e dezoito reais e vinte ..."
3,004919,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,Protesto Efetivo Maio/2025,NELSON HERMÓGENES DE MEDEIROS FREIRE (CPF: 067...,4788.4772,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,1,004919/2024,"R$ 4.788,48 (quatro mil, setecentos e oitenta ..."
4,000407,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,Protesto Efetivo Maio/2025,FELIPE FERREIRA MENEZES ARAUJO (CPF: 09008572427),2464.0757,PREFEITURA MUNICIPAL DE LAJES,1,000407/2025,"R$ 2.464,08 (dois mil, quatrocentos e sessenta..."
5,000409,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,Protesto Efetivo Maio/2025,CLÉCIO DA CÂMARA AZEVEDO (CPF: 30806062487),3416.2796,CONSÓRCIO INTERMUNICIPAL MULTIFINALITARIO POTI...,1,000409/2025,"R$ 3.416,28 (três mil, quatrocentos e dezessei..."
10,000410,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,Protesto Efetivo Maio/2025,SAULO ADRIANO FERREIRA DA CRUZ (CPF: 56557930400),15675.1632,CÂMARA MUNICIPAL DE BAIA FORMOSA,1,000410/2025,"R$ 15.675,16 (quinze mil, seiscentos e setenta..."
11,000411,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,Protesto Efetivo Maio/2025,FRANCISCO DANTAS DE ARAUJO (CPF: 36912247415),5442.7651,PREFEITURA MUNICIPAL DE TENENTE LAURENTINO CRUZ,1,000411/2025,"R$ 5.442,77 (cinco mil, quatrocentos e quarent..."
12,000413,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,Protesto Efetivo Maio/2025,JOSÉ GILDENOR DA FONSECA (CPF: 02203369469),6743.1792,PREFEITURA MUNICIPAL DE TRIUNFO POTIGUAR,1,000413/2025,"R$ 6.743,18 (seis mil, setecentos e quarenta e..."
13,000415,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,Protesto Efetivo Maio/2025,João Maria Montenegro da Silva (CPF: 48148750459),30286.5061,PREFEITURA MUNICIPAL DE SAO BENTO DO NORTE,1,000415/2025,"R$ 30.286,51 (trinta mil, duzentos e oitenta e..."


In [22]:
for desconto in descontos_aberto.itertuples():
    doc = DocxTemplate("templates/desconto_folha.docx")
    context = {
        'processo': desconto.processo,
        'assunto': desconto.assunto,
        'nome': desconto.nome,
        'orgao': desconto.nome_orgao,
        'valor': desconto.valor
    }
    nome = desconto.nome.split()[0].lower()
    processo_underline = desconto.processo.replace('/', '_')

    doc.render(context)
    d = f"saidas/desconto_folha/{processo_underline}_{nome}.docx"
    doc.save(d)
    generate_pdf_office(d, "saidas/desconto_folha/")
    print(f"Gerado arquivo para o processo {desconto.processo}")

Gerado arquivo para o processo 000730/2025
Gerado arquivo para o processo 000730/2025
Gerado arquivo para o processo 000730/2025
Gerado arquivo para o processo 004919/2024
Gerado arquivo para o processo 000407/2025
Gerado arquivo para o processo 000409/2025
Gerado arquivo para o processo 000410/2025
Gerado arquivo para o processo 000411/2025
Gerado arquivo para o processo 000413/2025
Gerado arquivo para o processo 000415/2025
Gerado arquivo para o processo 000724/2025
Gerado arquivo para o processo 000728/2025
Gerado arquivo para o processo 000729/2025
Gerado arquivo para o processo 000858/2025
Gerado arquivo para o processo 000858/2025
Gerado arquivo para o processo 000858/2025
Gerado arquivo para o processo 000861/2025
Gerado arquivo para o processo 000862/2025
Gerado arquivo para o processo 000866/2025
Gerado arquivo para o processo 000873/2025
Gerado arquivo para o processo 000877/2025
Gerado arquivo para o processo 000879/2025
Gerado arquivo para o processo 001480/2025
Gerado arqu